In [3]:
import os

root = "../../../"
import sys
sys.path.append(root)

from ember import PEFeatureExtractor
import numpy as np
import pickle
from copy import deepcopy

## Feature vector example

In [2]:
def get_ember_feature_vector(file_data):
    extractor = PEFeatureExtractor(feature_version=2, print_feature_warning=False)
    features = np.array(extractor.feature_vector(file_data), dtype=np.float32)
    return features

In [5]:
LIMIT = 10

pe_trainset_path = root + "data/archives/pe_trainset/PeX86Exe/"
samples = [os.path.join(pe_trainset_path + "backdoor", x) for x in os.listdir(pe_trainset_path + "backdoor")[0:LIMIT]]

with open(samples[0], "rb") as f:
    bytez = f.read()

In [10]:
feature_example = get_ember_feature_vector(bytez)

In [34]:
print(feature_example.shape)
feature_example

(2381,)


array([0.20424816, 0.0060663 , 0.00510061, ..., 0.        , 0.        ,
       0.        ], dtype=float32)

# Getting static sets

In [67]:
set_path = root + "data/train_val_test_sets/"
train_set = pickle.load(open(set_path+"X_train.pickle.set", "rb"))
y_train_set = np.load(set_path + "y_train.arr")
val_set = pickle.load(open(set_path+"X_val.pickle.set", "rb"))
y_val_set = np.load(set_path + "y_val.arr")

In [26]:
print(len(train_set))
train_set[0:3]

72684


['976c215002b82debe3023a8e5e2c9f88714a9daa0f180151c66e6def19973f62',
 '392bfa2256e5df20aa10dba3d9c8568906feb7cdec311c8bd4b5dee650b87931',
 '2fc9774dedd39618d8bcd5bc3814707c4e705bc5afacda7be740634e31ddc8c3']

In [27]:
print(y_train_set.shape)
y_train_set[0:3]

(72684,)


array([1., 1., 1.])

# Featurizing

In [74]:
def get_ember_feature_vector(file_data):
    extractor = PEFeatureExtractor(feature_version=2, print_feature_warning=False)
    features = np.array(extractor.feature_vector(file_data), dtype=np.float32)
    return features

set_path = root + "data/train_val_test_sets/"
train_set = pickle.load(open(set_path+"X_train.pickle.set", "rb"))
y_train_set = np.load(set_path + "y_train.arr")
val_set = pickle.load(open(set_path+"X_val.pickle.set", "rb"))
y_val_set = np.load(set_path + "y_val.arr")

pe_trainvalset_path = root + "data/archives/pe_trainset/PeX86Exe/"

X_ember_trainset = np.empty((0,2381))
X_ember_valset = np.empty((0,2381))
y_ember_trainset = []
y_ember_valset = []

malicious_folders = ["backdoor", "coinminer"]#, "dropper", "keylogger", "ransomware", "rat", "trojan"]
for folder in malicious_folders + ["clean"]:
    hashes = os.listdir(pe_trainvalset_path + folder)
    if len(hashes) == 0:
        print(f"\nSomething is wrong with {folder}\n")
    
    fullpaths = [os.path.join(pe_trainvalset_path + folder, x) for x in hashes]

    l = len(hashes)
    print(folder, end=f" (total={l}): |")
    for i in range(3): #l): # TBD 
        print(i, end="|")
        hh = hashes[i]
        fhh = fullpaths[i]    
        with open(fhh, "rb") as f:
            bytez = f.read()
    
        feature_vector = get_ember_feature_vector(bytez)
        
        if hh in train_set:
            X_ember_trainset = np.vstack([X_ember_trainset, feature_vector])
            
            if folder in malicious_folders:
                y_ember_trainset.append(1)
            else:
                y_ember_trainset.append(0)
            
            del(train_set[train_set.index(hh)])
        
        elif hh in val_set:
            X_ember_valset = np.vstack([X_ember_valset, feature_vector])
            
            if folder in malicious_folders:
                y_ember_valset.append(1)
            else:
                y_ember_valset.append(0)
            
            del(val_set[val_set.index(hh)])

        else:
            print(f"\nCannot find {fhh}")

    print()

backdoor (total=11088): |0|1|2|
coinminer (total=9518): |0|
Something is wrong with ../../data/archives/pe_trainset/PeX86Exe/coinminer\000061537fc1afe9f356fc3cfed13581524c3d49be789a91c8c3f836ff16805f
1|2|


In [ ]:
folder = "clean"
hashes = os.listdir(pe_trainvalset_path + folder)
fullpaths = [os.path.join(pe_trainvalset_path + folder, x) for x in hashes]
l = len(hashes)
print(folder, end=f" (total={l}): |")
for i in range(l):
    hh = hashes[i]
    fhh = fullpaths[i]